In [1]:
import datetime
import time
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorboard.plugins.hparams import api as hp
from sklearn.model_selection import train_test_split

import pandas as pd
from tqdm import tqdm
import os
import shutil

print(tf.__version__)
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    #device = tf.config.list_physical_devices('GPU')
    #tf.config.experimental.set_memory_growth(device[0], True)
    #tf.config.experimental.set_virtual_device_configuration(device[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
else:
    print("Please install GPU version of TF")

2.3.0
Default GPU Device: /device:GPU:0


In [2]:


'''
from azureml.core import Workspace, Dataset

subscription_id = '99ec244e-ed7b-4393-b86a-6805a53bd4f3'
resource_group = 'mvp-speaking'
workspace_name = 'datascience'

workspace = Workspace(subscription_id, resource_group, workspace_name)

train_dataset = Dataset.get_by_name(workspace, name='training')
validation_dataset = Dataset.get_by_name(workspace, name='validation')
'''

HP_NUM_UNITS=hp.HParam('num_units', hp.Discrete([128, 256, 512]))
HP_DROPOUT=hp.HParam('dropout', hp.RealInterval(0.1, 0.4))
HP_LEARNING_RATE= hp.HParam('learning_rate', hp.Discrete([0.001, 0.0005, 0.0001]))
HP_OPTIMIZER=hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

METRIC_ACCURACY='accuracy'

log_dir ='logs/hparam_tuning'
model_dir ='models'

with tf.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT,  HP_OPTIMIZER, HP_LEARNING_RATE],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )



In [3]:
EPOCHS = 10
embed_size = 512
polarity = ['Left','Left Leaning', 'Center','Left Leaning','Left']

In [4]:
# Load data
train_df = pd.read_csv('./articles_train.tsv', index_col=False, sep='\t')
#train_df = train_dataset.to_pandas_dataframe()
print(train_df.shape)
validate_df = pd.read_csv('./articles_validate.tsv', index_col=False, sep='\t')
print(validate_df.shape)
#validate_df = validation_dataset.to_pandas_dataframe()

train_X1 = train_df['title'].tolist()
train_X1 = np.array(train_X1, dtype=object)[:, np.newaxis]
train_X2 = train_df['content'].tolist()
train_X2 = np.array(train_X2, dtype=object)[:, np.newaxis]

train_X = [train_X1,train_X2]
train_y = np.asarray(pd.get_dummies(train_df.polarity), dtype = np.int8)

validate_X1 = validate_df['title'].tolist()
validate_X1 = np.array(validate_X1, dtype=object)[:, np.newaxis]
validate_X2 = validate_df['content'].tolist()
validate_X2 = np.array(validate_X2, dtype=object)[:, np.newaxis]

validate_X = [validate_X1,validate_X2]
validate_y = np.asarray(pd.get_dummies(validate_df.polarity), dtype = np.int8)


(320919, 11)
(76218, 11)


In [5]:
print()

In [6]:
def train_test_model(hparams, model_path):
    x1 = tf.keras.layers.Input(shape=[], dtype=tf.string, name ='title')
    embedding1 = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', trainable=False)(x1)
    #embedding1 = tf.keras.layers.Lambda(UniversalEmbedding,output_shape=(embed_size,))(x1)

    x2 = tf.keras.layers.Input(shape=[], dtype=tf.string, name ='content')
    embedding2 = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', trainable=False)(x2)
    #embedding2 = tf.keras.layers.Lambda(UniversalEmbedding,output_shape=(embed_size,))(x2)
    x = tf.keras.layers.concatenate([embedding1,embedding2])
    
    x = tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu)(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    x = tf.keras.layers.BatchNormalization()(x)
    #x = tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.Dense(64, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(x)
    x = tf.keras.layers.BatchNormalization()(x)
    output = tf.keras.layers.Dense(len(polarity),activation='sigmoid',name='output')(x)
    model = tf.keras.models.Model(inputs=[x1,x2], outputs=output)
    
    #setting the optimizer and learning rate
    optimizer = hparams[HP_OPTIMIZER]
    learning_rate = hparams[HP_LEARNING_RATE]
    if optimizer == "adam":
        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer=='rmsprop':
        optimizer = tf.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("unexpected optimizer name: %r" % (optimizer,))
        
    # Comiple the mode with the optimizer and learninf rate specified in hparams
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                     factor=0.1, 
                                                     patience=2, 
                                                     min_lr=1e-7, 
                                                     verbose=1)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    hp_callback = hp.KerasCallback(log_dir, hparams)
    history = model.fit(train_X, train_y, epochs=EPOCHS, 
            validation_data=(validate_X, validate_y),
            batch_size=32,
            callbacks=[reduce_lr,tensorboard_callback, hp_callback], 
            verbose=1, 
            use_multiprocessing=True, workers=16
            )
    print('model fitted.........')
    print('saving model.........' + model_path)
    model.save_weights(model_path)
    return history.history['val_accuracy'][-1]


def run(run_dir, hparams, model_name):
    model_path = '{}/{}'.format(model_dir, model_name)
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, model_path)
        #converting to tf scalar
        accuracy= tf.reshape(tf.convert_to_tensor(accuracy), []).numpy()
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [7]:
def main():
    shutil.rmtree(log_dir, ignore_errors=True)
    shutil.rmtree(model_dir, ignore_errors=True)
    os.mkdir(model_dir)
    session_num = 0
    
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
            for optimizer in HP_OPTIMIZER.domain.values:
                for learning_rate in HP_LEARNING_RATE.domain.values:
                    hparams = {
                        HP_NUM_UNITS: num_units,
                        HP_DROPOUT: dropout_rate,
                        HP_OPTIMIZER: optimizer,
                        HP_LEARNING_RATE: learning_rate,
                    }
                    model_name = "{}-{}-{}-{}-{}.h5".format(session_num,num_units,dropout_rate,optimizer,learning_rate)
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    try:
                        run('logs/hparam_tuning/' + run_name, hparams, model_name)
                    except Exception as e:
                        print(str(e))
                    session_num += 1


In [8]:
    start_time = datetime.datetime.now()
    print(start_time)
    main()
    print(datetime.datetime.now() - start_time)

2021-07-07 17:16:13.341140
--- Starting trial: run-0
{'num_units': 128, 'dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.0001}
Epoch 1/10
    1/10029 [..............................] - ETA: 0s - loss: 1.9309 - accuracy: 0.2500WARNING:tensorflow:From /anaconda/envs/azureml_py38/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


    2/10029 [..............................] - ETA: 1:38:55 - loss: 1.8766 - accuracy: 0.2656WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0664s vs `on_train_batch_end` time: 1.1172s). Check your callbacks.


 8628/10029 [========================>.....] - ETA: 2:08 - loss: 1.3903 - accuracy: 0.4624

KeyboardInterrupt: 